In [2]:
# !pip install requests
# !pip install beautifulsoup4

In [8]:
from bs4 import BeautifulSoup
import os
import requests
import time

DIRECTORY_PATH = './pages_scrape'
DIRECTORY_TO_SAVE = './code_scrapped'

def get_links_from_file(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()

    soup = BeautifulSoup(content, 'html.parser')
    anchors = soup.find_all('a', attrs={'data-testid': 'link-to-search-result'})
    
    return [anchor['href'] for anchor in anchors]

def github_link_to_raw(link):
    parts = link.split('/')
    username = parts[3]
    repository = parts[4]
    commit_hash = parts[6]
    file_path = '/'.join(parts[7:]).split('#')[0]  # Remove line number part
    raw_url = f"https://raw.githubusercontent.com/{username}/{repository}/{commit_hash}/{file_path}"
    return raw_url

def download_and_save(link, directory):
    raw_url = github_link_to_raw(link)
    response = requests.get(raw_url)
    if response.status_code != 200:
        print(f"Failed to download {raw_url}. Status code: {response.status_code}")
        return
    
    # Ensure directory exists
    os.makedirs(directory, exist_ok=True)

    # Save the content
    filename = os.path.join(directory, raw_url.split('/')[-1])
    with open(filename, 'wb') as file:
        file.write(response.content)

all_links = []

for file in os.listdir(DIRECTORY_PATH):
    if file.endswith('.html'):
        all_links.extend(get_links_from_file(os.path.join(DIRECTORY_PATH, file)))

for link in all_links:
    download_and_save(link, DIRECTORY_TO_SAVE)


In [10]:
len(all_links)

100

In [14]:
import time
from tqdm import tqdm

for link in tqdm(all_links):
    download_and_save(link, DIRECTORY_TO_SAVE)
    time.sleep(1)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [02:44<00:00,  1.64s/it]
